In [8]:
%%capture
!pip install boto3

In [1]:
import boto3
import random
import os
from dotenv import load_dotenv

load_dotenv()

AWS_ACCESS_KEY_ID = os.getenv('AWS_ACCESS_KEY_ID')
AWS_SECRET_ACCESS_KEY = os.getenv('AWS_SECRET_ACCESS_KEY')
REGION_NAME = os.getenv('REGION_NAME')
AGENT_ID = os.getenv('AGENT_ID')
AGENT_ALIAS_ID = os.getenv('AGENT_ALIAS_ID')

bedrock_agent_runtime = boto3.client(
                'bedrock-agent-runtime',
                aws_access_key_id = AWS_ACCESS_KEY_ID,
                aws_secret_access_key = AWS_SECRET_ACCESS_KEY,
                region_name= REGION_NAME
            )
session = random.randint(0,9999) #setting a random sessionID

## Below shows how to call the agents that are created in AWS bedrock

In [2]:
## Function for Invoking the agent (lead_gen) that is created in AWS bedrock

def generate_response_query(input_text,agent_id,agent_alias_id,session_id):
  response = bedrock_agent_runtime.invoke_agent(inputText=input_text,
      agentId=agent_id,
      agentAliasId=agent_alias_id,
      sessionId=session_id,
  )

  print(response.get('completion'))

  # Response returned from the agent

  event_stream = response.get('completion')

  for event in event_stream:
    if 'chunk' in event:
      data = event['chunk']['bytes']
      print(f"Final answer ->\n{data.decode('utf8')}")

In [3]:
input_text:str = "Create a lead as Mahesh Yadav, California, 339111222333" # replace this with a prompt relevant to your agent
agent_id:str = AGENT_ID # note this from the agent console on Bedrock
agent_alias_id:str = AGENT_ALIAS_ID # fixed for draft version of the agent
session_id:str =  str(session) # random identifier

In [4]:
generate_response_query(input_text,agent_id,agent_alias_id,session_id)

Final answer ->
Lead created successfully for Mahesh Yadav in California with phone number 339111222333.


## Creating an Agent and Invoking the Lambda functions stored in AWS

In [5]:
# Note: Here bedrock-agent is used instead of bedrock-agent-runtime
# Bedrock-agent has create_agent() function, while bedrock-agent-runtime has invoke_agent()

bedrock_agent = boto3.client(
                'bedrock-agent',
                aws_access_key_id = AWS_ACCESS_KEY_ID,
                aws_secret_access_key = AWS_SECRET_ACCESS_KEY,
                region_name= REGION_NAME
            )

In [6]:
aws_agent = bedrock_agent.create_agent(
    agentName='Lead_gen_notebook',
    description='Extract name, phone number and address for lead generation. Or extract address and monthly bill for calculating solar potential',
    foundationModel='anthropic.claude-v2',
    instruction="""You are a solar panel financial advisor, your job is to extract address and monthly bills for calling functions to calculate solar potential. You also are a solar panel lead generation employee, your job is to extract personal details like name, phone number and address for returning as response for calling functions to generate leads. Strictly respond to the question for calling the function that is required to answer.""",
    agentResourceRoleArn='arn:aws:iam::674564987265:role/service-role/AmazonBedrockExecutionRoleForAgents_KT4F6CHPJV'
)

In [7]:
#Creating the action group for the agent
response = bedrock_agent.create_agent_action_group(
    agentId=aws_agent['agent'].get('agentId'),
    agentVersion="DRAFT",
    actionGroupName='LeadGenAction',
    actionGroupExecutor={
        'lambda': 'arn:aws:lambda:us-east-1:674564987265:function:lead_gen_solar'
    },
    apiSchema={
        's3': {
            's3BucketName': 'legalgraphbucket',
            's3ObjectKey': 'lead_generation.json'
        }
    },
    actionGroupState='ENABLED'
)
print(response)

{'ResponseMetadata': {'RequestId': 'bffbde3d-3d85-4cd2-b886-8c312a49dcc5', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Fri, 15 Dec 2023 13:29:58 GMT', 'content-type': 'application/json', 'content-length': '439', 'connection': 'keep-alive', 'x-amzn-requestid': 'bffbde3d-3d85-4cd2-b886-8c312a49dcc5', 'x-amz-apigw-id': 'P_IxTHZpIAMEkQg=', 'x-amzn-trace-id': 'Root=1-657c54d4-6541f97415644b82307ad7d2'}, 'RetryAttempts': 0}, 'agentActionGroup': {'agentId': 'PANULRHLZG', 'agentVersion': 'DRAFT', 'actionGroupId': 'ODHW14PR6L', 'actionGroupName': 'LeadGenAction', 'createdAt': datetime.datetime(2023, 12, 15, 13, 29, 58, 314933, tzinfo=tzutc()), 'updatedAt': datetime.datetime(2023, 12, 15, 13, 29, 58, 314933, tzinfo=tzutc()), 'actionGroupExecutor': {'lambda': 'arn:aws:lambda:us-east-1:674564987265:function:lead_gen_solar'}, 'apiSchema': {'s3': {'s3BucketName': 'legalgraphbucket', 's3ObjectKey': 'lead_generation.json'}}, 'actionGroupState': 'ENABLED'}}


In [8]:
# Check status of the agent

response = bedrock_agent.get_agent(
    agentId=aws_agent['agent'].get('agentId')
)
print(response['agent'].get('agentStatus'))

NOT_PREPARED


In [9]:
#Preparing the agent

response = bedrock_agent.prepare_agent(
    agentId=aws_agent['agent'].get('agentId')
)

In [10]:
# Check status of the agent again

response = bedrock_agent.get_agent(
    agentId=aws_agent['agent'].get('agentId')
)
print(response['agent'].get('agentStatus'))

PREPARED


In [11]:
# Creating the alias
agent_alias = bedrock_agent.create_agent_alias(
    agentId=aws_agent['agent'].get('agentId'),
    agentAliasName='Dummy',
)
print(agent_alias)

{'ResponseMetadata': {'RequestId': '141468e0-b7c0-49eb-96f1-29829f343809', 'HTTPStatusCode': 202, 'HTTPHeaders': {'date': 'Fri, 15 Dec 2023 13:30:09 GMT', 'content-type': 'application/json', 'content-length': '329', 'connection': 'keep-alive', 'x-amzn-requestid': '141468e0-b7c0-49eb-96f1-29829f343809', 'x-amz-apigw-id': 'P_IzSFvoIAMENyA=', 'x-amzn-trace-id': 'Root=1-657c54e1-28b975d373d88aa1283a98ef'}, 'RetryAttempts': 0}, 'agentAlias': {'agentId': 'PANULRHLZG', 'agentAliasId': 'UQTAYEPG0W', 'agentAliasName': 'Dummy', 'agentAliasArn': 'arn:aws:bedrock:us-east-1:674564987265:agent-alias/PANULRHLZG/UQTAYEPG0W', 'routingConfiguration': [], 'createdAt': datetime.datetime(2023, 12, 15, 13, 30, 9, 692477, tzinfo=tzutc()), 'updatedAt': datetime.datetime(2023, 12, 15, 13, 30, 9, 692477, tzinfo=tzutc()), 'agentAliasStatus': 'CREATING'}}


In [13]:
#Preparing the agent

response = bedrock_agent.prepare_agent(
    agentId=aws_agent['agent'].get('agentId')
)

In [14]:
input_text:str = "Create a lead as Mahesh Yadav, California, 339111222333" # replace this with a prompt relevant to your agent

input_text2:str = "Calculate solar potential for 9813 Sherborne Ave, Bakersfield, CA 93311, USA with monthly bill as 300 USD"


agent_id:str = aws_agent['agent'].get('agentId') # note this from the agent console on Bedrock
agent_alias_id:str = agent_alias['agentAlias'].get('agentAliasId') # fixed for draft version of the agent
session_id:str =  str(session) # session ID

In [15]:
generate_response_query(input_text,agent_id,agent_alias_id,session_id)

Final answer ->
The lead has been created successfully for Mahesh Yadav in California with phone number 339111222333.


In [16]:
generate_response_query(input_text2,agent_id,agent_alias_id,session_id)

Final answer ->
Here is the solar potential analysis for the address 9813 Sherborne Ave, Bakersfield, CA 93311, USA with a monthly electricity bill of $300 USD:

- Estimated annual solar production: 13,199 kWh/year  
- Remaining lifetime utility bill: $7,857
- Federal incentive: $6,837
- Solar percentage: 99.15%
- Leasing savings:
  - Annual leasing cost: $1,544
  - 20 year savings: $51,227
  - Lifetime savings: $51,227
- Cash purchase savings:  
  - Payback period: 5.25 years
  - 20 year savings: $82,126
  - Lifetime savings: $82,126
- Financed purchase savings:
  - Annual loan payment: $1,544
  - 20 year savings: $51,227
  - Lifetime savings: $51,227

In summary, installing solar panels is financially viable for this location based on the analysis. Leasing, cash purchase, or financed purchase are all good options to consider further.
